In [1]:
!pip install kagglehub
import kagglehub
path = kagglehub.dataset_download("phhasian0710/seed-iv")
print("Path to dataset files:", path)

100%|██████████| 13.7G/13.7G [02:39<00:00, 92.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1


In [2]:
import os
for root, dirs, files in os.walk(path):
    for dir in dirs:
        print(os.path.join(root, dir))  # Print directories
    for file in files:
        print(os.path.join(root, file))  # Print files

/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eye_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eye_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/ReadMe.txt
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/Channel Order.xlsx
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eeg_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eeg_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eye_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eye_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/ReadMe.txt
/root/.cache/ka

In [3]:
!pip install tensorflow
import numpy as np
import tensorflow as tf
import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras import activations

In [4]:
import os
import numpy as np
from scipy.io import loadmat
directories=["/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/{}".format(i+1) for i in range(3)]
print(directories)

['/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/1', '/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/2', '/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/3']


In [5]:
# EEG channel coordinates
channel_coords = [
    ['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'],
    ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'],
    ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'],
    ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'],
    ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'],
    ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'],
    ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'],
    ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']
]

In [6]:
channel_list = [
    'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ',
    'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2',
    'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4',
    'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4',
    'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6',
    'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1',
    'O1', 'OZ', 'O2', 'CB2'
]
# Verifying the length of channel coordinates
print(len(channel_coords), len(channel_coords[0]))

8 9


In [7]:
# Creating a dictionary to map channel names to coordinates
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if channel_list[n] == x:
                coord_dict[n] = (i, j)
print(coord_dict)

{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}


In [8]:
# Initialize the array with the correct shape
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories), len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64))

In [9]:
# Iterate over directories to load and process data
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(os.path.join(dire, file)) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
      for j in range(n):
            for k, key in enumerate(perSample):
                # Transpose and pad the sample
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0, 2, 1))
                sample = np.pad(sample, [(0, 0), (0, 0), (0, 64 - sample.shape[2])])
                # Fill the array based on channel locations
                for l, channel in enumerate(sample):
                  array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

# Output the shape of the final array
print(array.shape)

/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/1
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/2
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/3
(3, 15, 24, 4, 8, 9, 5, 64)


In [10]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)
X_loso = array[:,:,:,1,:,:,]
X_loso = np.transpose(X_loso, (0,1,2,6,3,4,5))
print(X_loso.shape)

(1080, 4, 8, 9, 5, 64)
(3, 15, 24, 64, 8, 9, 5)


In [11]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}
y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)
print(y.shape)
y_loso = np.reshape(y, (3,15,24))
print(y_loso.shape)

(1080,)
(3, 15, 24)


In [12]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4,activation='softmax')
])

In [13]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [14]:
def LSTM():
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
    ])
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [17]:
def convlstm2d():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(16, kernel_size=5, padding="same", return_sequences = False, input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(10))
    return model

In [18]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = fully_conv()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_separable_conv.py:104: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.2813 - loss: 2660.7061 - val_accuracy: 0.2824 - val_loss: 340.2416
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.3162 - loss: 1580.1527 - val_accuracy: 0.3148 - val_loss: 18.9671
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.3510 - loss: 68.8944 - val_accuracy: 0.3611 - val_loss: 10.9775
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 124ms/step - accuracy: 0.3548 - loss: 83.6672 - val_accuracy: 0.3657 - val_loss: 4313.7051
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.3627 - loss: 174.8836 - val_accuracy: 0.2917 - val_loss: 777.0311
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.4526 - loss: 15.1511 - val_accuracy: 0.3194 - val_loss: 1556.4722
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.3622 - loss: 13.8414 - val_accuracy: 0.3380 - val_loss: 5004.6191
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 148ms/step - accuracy: 0.4663 - loss: 3.5984 - v

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ separable_conv1d (SeparableConv1D)   │ (None, 64, 128)             │         188,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 32, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_1 (SeparableConv1D) │ (None, 32, 128)             │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 16, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_2 (SeparableConv1D) │ (None, 16, 128)             │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_3 (SeparableConv1D) │ (None, 8, 128)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 4, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 916,910 (3.50 MB)

 Trainable params: 305,636 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 611,274 (2.33 MB)

In [20]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = single_conv()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.2419 - loss: 137330224.0000 - val_accuracy: 0.3102 - val_loss: 7783634.5000
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.4083 - loss: 35300676.0000 - val_accuracy: 0.2546 - val_loss: 153832064.0000
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.4203 - loss: 5100375.5000 - val_accuracy: 0.3750 - val_loss: 129459184.0000
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.5075 - loss: 305598.6875 - val_accuracy: 0.3796 - val_loss: 109700392.0000
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.4966 - loss: 250314.6406 - val_accuracy: 0.2778 - val_loss: 481880352.0000
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.4218 - loss: 2867912.7500 - val_accuracy: 0.2963 - val_loss: 153835104.0000
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.3813 - loss: 6903568.5000 - val_accuracy: 0.3796 - val_loss: 270997344.0000
Epoch 8/50
27

In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 62, 32)              │         138,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 31, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 992)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         127,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 808,910 (3.09 MB)

 Trainable params: 269,636 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 539,274 (2.06 MB)

In [22]:
model = LSTM()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 268ms/step - accuracy: 0.2169 - loss: 2.0800 - val_accuracy: 0.2037 - val_loss: 1.4943
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.3006 - loss: 1.3959 - val_accuracy: 0.3102 - val_loss: 1.4189
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 230ms/step - accuracy: 0.3679 - loss: 1.3109 - val_accuracy: 0.3935 - val_loss: 1.3455
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 261ms/step - accuracy: 0.4488 - loss: 1.2308 - val_accuracy: 0.3981 - val_loss: 1.3674
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 213ms/step - accuracy: 0.4489 - loss: 1.1983 - val_accuracy: 0.3750 - val_loss: 1.3682
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 214ms/step - accuracy: 0.4334 - loss: 1.2006 - val_accuracy: 0.3796 - val_loss: 1.3320
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 248ms/step - accuracy: 0.4520 - loss: 1.1489 - val_accuracy: 0.3981 - val_loss: 1.3402
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 227ms/step - accuracy: 0.4392 - loss: 1.1482 - val_accurac

In [23]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (32, 100)                   │         616,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (32, 128)                   │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (32, 10)                    │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,891,856 (7.22 MB)

 Trainable params: 630,618 (2.41 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,261,238 (4.81 MB)

In [24]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,1]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 5)


In [28]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,:]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = convlstm2d()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

(1080, 64, 8, 9, 20)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 95s 3s/step - accuracy: 0.2915 - loss: 1.7741 - val_accuracy: 0.3889 - val_loss: 1.3940
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - accuracy: 0.4593 - loss: 1.2693 - val_accuracy: 0.3611 - val_loss: 1.3842
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.4538 - loss: 1.2205 - val_accuracy: 0.3704 - val_loss: 1.3251
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.4646 - loss: 1.1649 - val_accuracy: 0.3009 - val_loss: 1.3571
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.4590 - loss: 1.1512 - val_accuracy: 0.3657 - val_loss: 1.3714
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.5002 - loss: 1.0417 - val_accuracy: 0.3380 - val_loss: 1.4977
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.5563 - loss: 1.0009 - val_accuracy: 0.3657 - val_loss: 1.2490
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.5025 - loss: 1.0421 - val_accuracy: 0.3750 - val_los

In [29]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = dense()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 220ms/step - accuracy: 0.2852 - loss: 162023824.0000 - val_accuracy: 0.1852 - val_loss: 924094848.0000
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 207ms/step - accuracy: 0.3136 - loss: 71489056.0000 - val_accuracy: 0.2731 - val_loss: 4284506.0000
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 211ms/step - accuracy: 0.3086 - loss: 5199468.5000 - val_accuracy: 0.2454 - val_loss: 4938272.0000
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.3193 - loss: 29661182.0000 - val_accuracy: 0.2454 - val_loss: 4679768.0000
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.3089 - loss: 155872.0938 - val_accuracy: 0.2454 - val_loss: 4595071.5000
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 223ms/step - accuracy: 0.3057 - loss: 599.4595 - val_accuracy: 0.2454 - val_loss: 4435610.0000
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 257ms/step - accuracy: 0.2990 - loss: 156.7281 - val_accuracy: 0.2454 - val_loss: 4396330.0000
Epoch 8/20
27/27 ━━━